In [23]:
import torch
import transformers
from transformers import AutoModelWithLMHead, PreTrainedTokenizerFast
from fastai.text.all import *
import re
import fastai

print(torch.__version__)
print(transformers.__version__)
print( fastai.__version__)

1.8.0
4.20.1
2.7.7


In [24]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>') 
model = AutoModelWithLMHead.from_pretrained("skt/kogpt2-base-v2")
text = """ 옛날 옛날 어느 마을에 흥부와 놀부 형제가 """
input_ids = tokenizer.encode(text)
gen_ids = model.generate(torch.tensor([input_ids]),
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True
                        )
generated = tokenizer.decode(gen_ids[0,:].tolist())
print(generated)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
c:\ProgramData\Anaconda3\envs\readvice\lib\site-packages\transformers\models\auto\modeling_auto.py:969: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


옛날 옛날 어느 마을에 흥부와 놀부 형제가 왁자지껄 떠들어대며 "우리 집엔 왜 이렇게 많은 사람들이 모여 사는 거야?" 하고 묻는다.
그런데 그 마을 사람들은 모두 다들 자기네 동네에 살고 있는 사람들이라고 한다.
이렇게 해서 우리 마을은 '흥부가 살던 곳'이라는 뜻의 '고향'이 되었다.
그리고 이 고향은 바로 지금의 서울 종로구 숭인동이다.
숭인동은 원래 종로에서 가장 오래된 주택가였다.
1970년대까지만 해도 이곳은 재개발로 인해 헐리고 빈집이 많아졌다.
하지만 1980년대 들어 다시 활기를 되찾기 시작했다.
당시만 하더라도 이곳에는 낡은 건물들이 많이 남아 있었다.
그러나 1990년대 들어서부터는 예전의


In [4]:
# import pandas as pd
# import re
# from icecream import ic
# df = pd.read_csv('./data/book_report_data.csv', index_col=0)
# df.drop_duplicates(keep='first', inplace=True, ignore_index=False)
# df = df.to_csv('./data/book_report_data.txt', index=False)

In [1]:
from hanspell import spell_checker
import re
from pykospacing import Spacing
with open('./data/book_report_data.txt', 'r', encoding='utf-8') as f:
    data = f.read()
data=" ".join(data.split())
# print(len(data))
data = data.replace('\n|\t', ' ')
print(len(data))
new_sent = data.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기
print(len(new_sent))
spacing = Spacing()
kospacing_sent = spacing(new_sent) 
print(len(kospacing_sent))
# spelled_sent = spell_checker.check(data)
# print(len(spelled_sent))
# data = spelled_sent.checked
# print(len(data))
data = re.sub('[-=+,#/\:^$@*\"※~&%ㆍ』\\‘|\(\)\[\]\<\>`\'…》]','', data)
# print(len(data))
data = re.sub('[a-zA-Z]' , '', data)
print(len(data))


# UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 20: illegal multibyte sequence
# 해당 에러 ->  encoding='utf-8' 넣어 주기

2059672
1593742
2095744
2003417


In [15]:

#model input output tokenizer
class TransformersTokenizer(Transform):
   def __init__(self, tokenizer): self.tokenizer = tokenizer
   def encodes(self, x): 
       toks = self.tokenizer.tokenize(x)
       return tensor(self.tokenizer.convert_tokens_to_ids(toks))
   def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

#split data
train=data[:int(len(data)*0.9)]
test=data[int(len(data)*0.9):]
splits = [[0],[1]]

#init dataloader
tls = TfmdLists([train,test], TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
batch,seq_len = 8,256
dls = tls.dataloaders(bs=4, seq_len=seq_len)

In [17]:
#gpt2 ouput is tuple, we need just one val
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()
lr=learn.lr_find()
print(lr)
learn.fit_one_cycle(50, lr)
# learn.fine_tune(30)

KeyboardInterrupt: 

In [ ]:
prompt='여행가고 싶어지는'
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
preds = learn.model.generate(inp,
                           max_length=128,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           repetition_penalty=2.0,
                           use_cache=True
                          )
tokenizer.decode(preds[0].cpu().numpy())

'여행가고 싶어지는 로컬 마트에서 먹고 싶은 음식을 검색으로 찾아다니는 홈페이지이다.이 책의 저자 밥 프록터는 뉴욕에 있는 알베르게 추천여행지 중 한 곳을 약 5년 동안 여유롭게 다녀 온 스팟을 소개한다. 유명 관광지는 물론이고 다양한 볼거리 먹거리와 이색 장소까지 많은 매력을 가진다. 특히 세계적인 호텔 체인도 많이 들어와 있고 현지인과 함께 하는 술집이나 식당 쇼핑 명소도 소개되어 있다.저자는 현지에 친구들이 소개한  에서 만나도록 권유한다힘든 시간들을 어떻게 써야 할 지 막막했는데 지금이라도 이 책을 만나게 된다면 조금은 더 편안한 마음으로 여행을 즐길 수'

In [ ]:
learn.model.save_pretrained("./models/kogpt2_bookreport_backup50")